In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import IterableDataset, DataLoader

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

import hdbscan
from tqdm import tqdm
import matplotlib.pyplot as plt

# Seus módulos locais
from external_information import ExternalInformationFusionDTPC, ExternalInformationDense
from partial_information import CoordLSTM

In [ ]:
class ParquetCityDDataset(IterableDataset):
    def __init__(
        self,
        parquet_path: str,
        city_list: list[str],
        chunk_size: int = 10_000,
        sequence_length: int = 24,  # NOVO: janela temporal
        prediction_steps: int = 1    # NOVO: quantos passos predizer
    ):
        """
        Dataset que constrói sequências temporais para cada usuário.
        
        sequence_length: quantos timesteps passados usar (ex: 24 slots)
        prediction_steps: quantos timesteps futuros predizer (ex: 1)
        """
        self.parquet_path = parquet_path
        self.city_list = city_list
        self.city_set = set(city_list)
        self.city_to_idx = {c: i for i, c in enumerate(city_list)}
        self.chunk_size = chunk_size
        self.sequence_length = sequence_length
        self.prediction_steps = prediction_steps

    def _build_sequences_for_user(self, user_data):
        """
        Constrói sequências temporais para um usuário específico.
        user_data: DataFrame com colunas [uid, d, t, city, POI, x, y] ordenado por (d,t)
        """
        sequences = []
        
        for i in range(len(user_data) - self.sequence_length - self.prediction_steps + 1):
            # Janela de entrada (histórico)
            seq_start = i
            seq_end = i + self.sequence_length
            
            # Próximos passos (alvos)
            target_start = seq_end
            target_end = target_start + self.prediction_steps
            
            # Extrai sequência de coordenadas (input para LSTM)
            coords_seq = user_data.iloc[seq_start:seq_end][['x', 'y']].values
            
            # Extrai informações do último ponto da sequência (contexto atual)
            current_info = user_data.iloc[seq_end - 1]
            
            # Extrai coordenadas alvo
            target_coords = user_data.iloc[target_start:target_end][['x', 'y']].values
            
            sequences.append({
                'uid': current_info['uid'],
                'd': current_info['d'], 
                't': current_info['t'],
                'city_idx': self.city_to_idx[current_info['city']],
                'poi': current_info['POI'],
                'coords_seq': coords_seq.astype(np.float32),  # (sequence_length, 2)
                'target_coords': target_coords.astype(np.float32)  # (prediction_steps, 2)
            })
            
        return sequences

    def __iter__(self):
        pf = pq.ParquetFile(self.parquet_path)
        
        for batch in pf.iter_batches(batch_size=self.chunk_size):
            table = pa.Table.from_batches([batch], schema=pf.schema_arrow)

            # Filtra cidades
            mask = pc.is_in(table.column("city"), pa.array(list(self.city_set)))
            table = table.filter(mask)
            if table.num_rows == 0:
                continue

            # Converte para pandas para facilitar agrupamento
            df = table.to_pandas()
            
            # Normaliza POIs (CORREÇÃO IMPORTANTE)
            poi_cols = [col for col in df.columns if 'POI' in col or col == 'POI']
            if poi_cols:
                df[poi_cols] = np.log1p(df[poi_cols])  # log(1+x) para estabilizar
            
            # Agrupa por usuário e ordena por tempo
            df = df.sort_values(['uid', 'd', 't'])
            
            for uid, user_group in df.groupby('uid'):
                # Só processa usuários com dados suficientes
                if len(user_group) < self.sequence_length + self.prediction_steps:
                    continue
                    
                sequences = self._build_sequences_for_user(user_group)
                
                for seq in sequences:
                    yield (
                        torch.tensor(seq['uid'], dtype=torch.long),
                        torch.tensor(seq['d'], dtype=torch.long),
                        torch.tensor(seq['t'], dtype=torch.long),
                        torch.tensor(seq['city_idx'], dtype=torch.long),
                        torch.from_numpy(seq['poi']),
                        torch.from_numpy(seq['coords_seq']),     # (seq_len, 2)
                        torch.from_numpy(seq['target_coords'])   # (pred_steps, 2)
                    )

## Fusão de dados

In [3]:
class WeightedFusion(nn.Module):
    """
    Funde dois vetores de mesmo tamanho (B, dim) por uma soma ponderada:
        output = w_r * static_red + w_e * dyn_emb
    onde w_r e w_e são parâmetros escalar aprendíveis.
    A saída tem a mesma dimensão (dim) dos vetores de entrada.
    """
    def __init__(self, dim: int = 20, init_w_r: float = 0.5, init_w_e: float = 0.5):
        super().__init__()
        # pesos escalares aprendíveis
        self.w_r = nn.Parameter(torch.tensor(init_w_r, dtype=torch.float32))
        self.w_e = nn.Parameter(torch.tensor(init_w_e, dtype=torch.float32))
        self.dim = dim

    def forward(self, static_red: torch.Tensor, dyn_emb: torch.Tensor) -> torch.Tensor:
        """
        static_red: Tensor[B, dim] – vetor reduzido estático
        dyn_emb:     Tensor[B, dim] – vetor reduzido dinâmico (LSTM)
        retorna:     Tensor[B, dim] – fusão ponderada
        """
        # checa que as dimensões batem
        assert static_red.shape == dyn_emb.shape and static_red.size(1) == self.dim
        # soma ponderada
        fused = self.w_r * static_red + self.w_e * dyn_emb
        return fused

In [4]:
n_users_by_city = {"A":100_000, "B":25_000, "C":20_000, "D":6_000}

In [14]:
class MLP500(nn.Module):
    """
    MLP simples com 1 hidden layer de 500 ReLUs.
     - in_dim → 500 → C logits
    """
    def __init__(self, in_dim: int, hidden_dim: int, n_clusters: int):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, n_clusters)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, in_dim)
        x = F.relu(self.fc1(x))  # (B, hidden_dim)
        return self.fc2(x)       # (B, n_clusters)


class DestinationHead(nn.Module):
    """
    Combina um MLP500 + softmax + weighted sum pelos cluster centers.
    """
    def __init__(self,
                 in_dim: int,           # deve ser 20
                 hidden_dim: int,       # 500
                 cluster_centers: torch.Tensor  # (C,2)
    ):
        super().__init__()
        C, coord_dim = cluster_centers.shape
        assert coord_dim == 2
        self.mlp500 = MLP500(in_dim, hidden_dim, C)
        # armazenamos centros como buffer (não aprensíveis)
        self.register_buffer("centers", cluster_centers)

    def forward(self, fused: torch.Tensor) -> torch.Tensor:
        """
        fused: (B, 20) → retorna coords (B,2)
        """
        logits = self.mlp500(fused)        # (B, C)
        P      = F.softmax(logits, dim=1)  # (B, C)
        # média ponderada: P @ centers → (B,2)
        coords = P @ self.centers
        return coords

In [12]:
import hdbscan  

def compute_hdbscan_centers(
    parquet_path: str,
    city_letter: str = "D",
    day_threshold: int = 60,
    chunk_size: int = 50_000,
    min_cluster_size: int = 100
) -> torch.Tensor:
    """
    1) Lê em chunks o Parquet,
    2) filtra city==city_letter e d<day_threshold,
    3) acumula destinos (x,y),
    4) roda HDBSCAN para descobrir clusters,
    5) retorna tensor K×2 com os centros (média dos pontos em cada cluster).
    """
    # 1) coleta todos os destinos de treino da cidade
    pf = pq.ParquetFile(parquet_path)
    coords_list = []
    for batch in pf.iter_batches(batch_size=chunk_size):
        tbl = pa.Table.from_batches([batch], schema=pf.schema_arrow)
        mask = pc.and_(
            pc.equal(tbl.column("city"), city_letter),
            pc.less(tbl.column("d"), day_threshold)
        )
        tbl = tbl.filter(mask)
        if tbl.num_rows == 0:
            continue
        xs = tbl.column("x").to_numpy()
        ys = tbl.column("y").to_numpy()
        coords_list.append(np.stack([xs, ys], axis=1))
    coords = np.vstack(coords_list)  # shape (N,2)

    # 2) (opcional) amostra para acelerar
    if len(coords) > 200_000:
        idx = np.random.choice(len(coords), 200_000, replace=False)
        sample = coords[idx]
    else:
        sample = coords

    # 3) roda HDBSCAN
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        metric="euclidean",
        cluster_selection_method="eom"
    )
    labels = clusterer.fit_predict(sample)  # shape (M,)

    # 4) calcula centros como média de cada cluster
    unique_labels = [lab for lab in np.unique(labels) if lab >= 0]
    centers = []
    for lab in unique_labels:
        pts = sample[labels == lab]
        centers.append(pts.mean(axis=0))
    centers = np.vstack(centers)  # shape (K,2)

    return torch.from_numpy(centers.astype(np.float32))

In [ ]:
class HuMobModel(nn.Module):
    """
    Modelo completo que combina todas as partes e faz rollout para múltiplos passos.
    É como um "diretor de orquestra" que coordena todos os músicos (módulos).
    """
    def __init__(
        self,
        n_users: int,
        n_days: int,
        n_slots: int,
        n_cities: int,
        cluster_centers: torch.Tensor,
        emb_dim: int = 10,
        poi_in_dim: int = 85,
        poi_out_dim: int = 10,
        lstm_hidden: int = 10,
        fusion_dim: int = 20
    ):
        super().__init__()
        
        # Componentes que você já tem
        self.fusion = ExternalInformationFusionDTPC(
            n_users=n_users,
            n_days=n_days,
            n_slots=n_slots,
            n_cities=n_cities,
            emb_dim=emb_dim,
            poi_in_dim=poi_in_dim,
            poi_out_dim=poi_out_dim
        )
        self.dense = ExternalInformationDense(
            in_dim=self.fusion.out_dim, 
            out_dim=fusion_dim
        )
        self.lstm = CoordLSTM(
            input_size=2, 
            hidden_size=lstm_hidden, 
            bidirectional=True
        )
        self.weighted_fusion = WeightedFusion(dim=fusion_dim)
        self.destination_head = DestinationHead(
            in_dim=fusion_dim,
            hidden_dim=500,
            cluster_centers=cluster_centers
        )
        
    def forward_single_step(self, uid, d, t, city, poi, coords_seq):
        """
        Faz uma predição para um único passo.
        É como fazer uma "foto" da situação atual e prever o próximo movimento.
        """
        # Informação estática (contexto)
        static_emb = self.fusion(uid, d, t, city, poi)
        static_red = self.dense(static_emb)
        
        # Informação dinâmica (padrão de movimento)
        dyn_emb = self.lstm(coords_seq)
        
        # Fusão inteligente
        fused = self.weighted_fusion(static_red, dyn_emb)
        
        # Predição final
        pred_coords = self.destination_head(fused)
        
        return pred_coords
    
    def rollout_predictions(
        self, 
        uid, d, t, city, poi, coords_seq, 
        n_steps: int,
        use_predictions: bool = True
    ):
        """
        Faz predições para múltiplos passos futuros.
        É como um "GPS" que planeja toda a rota, não só o próximo movimento.
        
        use_predictions: se True, usa predições anteriores como input (autoregressivo)
                        se False, sempre usa a sequência original (teacher forcing)
        """
        predictions = []
        current_seq = coords_seq.clone()
        
        for step in range(n_steps):
            # Prediz próximo passo
            pred = self.forward_single_step(uid, d, t, city, poi, current_seq)
            predictions.append(pred)
            
            if use_predictions and step < n_steps - 1:
                # Atualiza sequência: remove primeiro ponto, adiciona predição
                # É como "deslizar uma janela" no tempo
                new_point = pred.unsqueeze(1)  # (batch, 1, 2)
                current_seq = torch.cat([current_seq[:, 1:, :], new_point], dim=1)
            
            # Incrementa tempo (simplificado - você pode ajustar conforme sua lógica)
            t = t + 1
            
        return torch.stack(predictions, dim=1)  # (batch, n_steps, 2)
    
    def forward(self, uid, d, t, city, poi, coords_seq, n_steps=1):
        """Forward principal - pode ser usado tanto para treino quanto inferência."""
        if n_steps == 1:
            return self.forward_single_step(uid, d, t, city, poi, coords_seq)
        else:
            return self.rollout_predictions(uid, d, t, city, poi, coords_seq, n_steps)


def discretize_coordinates(coords_pred: torch.Tensor, grid_size: int = 200):
    """
    Converte coordenadas contínuas para grid discreto [0, grid_size-1].
    É como "encaixar" as predições nas células da grade.
    """
    # Arredonda e garante que está no intervalo correto
    coords_discrete = torch.round(coords_pred).long()
    coords_discrete = torch.clamp(coords_discrete, 0, grid_size - 1)
    return coords_discrete


def create_submission_data(model, dataloader, device, n_prediction_days=15, slots_per_day=48):
    """
    Gera dados para submissão no formato exigido pelo HuMob.
    É como "traduzir" as predições do modelo para o formato que os juízes esperam.
    """
    model.eval()
    submission_rows = []
    
    with torch.no_grad():
        for batch in dataloader:
            uid, d, t, city, poi, coords_seq, _ = [b.to(device) for b in batch]
            
            # Total de passos a prever
            total_steps = n_prediction_days * slots_per_day
            
            # Faz rollout
            predictions = model.rollout_predictions(
                uid, d, t, city, poi, coords_seq, 
                n_steps=total_steps,
                use_predictions=True
            )
            
            # Discretiza coordenadas
            predictions_discrete = discretize_coordinates(predictions)
            
            # Converte para formato de submissão
            batch_size = uid.size(0)
            for b in range(batch_size):
                user_id = uid[b].item()
                start_day = d[b].item()
                
                for step in range(total_steps):
                    day = start_day + (step // slots_per_day)
                    slot = step % slots_per_day
                    x = predictions_discrete[b, step, 0].item()
                    y = predictions_discrete[b, step, 1].item()
                    
                    submission_rows.append({
                        'uid': user_id,
                        'day': day,
                        'slot': slot,
                        'x': x,
                        'y': y
                    })
    
    return pd.DataFrame(submission_rows)

In [ ]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss, MSELoss
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

class HuMobLoss(nn.Module):
    """
    Loss híbrida que combina classificação (qual cluster) + regressão (coordenadas exatas).
    É como ter dois "professores": um que ensina "em que região" e outro "exatamente onde".
    """
    def __init__(self, cluster_centers: torch.Tensor, alpha: float = 0.7):
        super().__init__()
        self.register_buffer('cluster_centers', cluster_centers)
        self.alpha = alpha  # peso para o loss de coordenadas
        self.ce_loss = CrossEntropyLoss()
        self.mse_loss = MSELoss()
        
    def forward(self, predicted_coords, target_coords, cluster_logits=None):
        """
        predicted_coords: (batch, 2) ou (batch, steps, 2)
        target_coords: (batch, 2) ou (batch, steps, 2) 
        cluster_logits: (batch, n_clusters) ou (batch, steps, n_clusters)
        """
        # Achata se for sequência
        if predicted_coords.dim() == 3:
            predicted_coords = predicted_coords.view(-1, 2)
            target_coords = target_coords.view(-1, 2)
            if cluster_logits is not None:
                cluster_logits = cluster_logits.view(-1, cluster_logits.size(-1))
        
        # Loss de coordenadas (regressão)
        coord_loss = self.mse_loss(predicted_coords, target_coords)
        
        total_loss = self.alpha * coord_loss
        
        # Loss de cluster (classificação) - opcional
        if cluster_logits is not None:
            # Encontra cluster mais próximo para cada target
            distances = torch.cdist(target_coords, self.cluster_centers)  # (batch, n_clusters)
            target_clusters = distances.argmin(dim=1)  # (batch,)
            
            cluster_loss = self.ce_loss(cluster_logits, target_clusters)
            total_loss += (1 - self.alpha) * cluster_loss
            
            return total_loss, coord_loss, cluster_loss
        
        return total_loss, coord_loss, torch.tensor(0.0)


def train_humob_model(
    model: HuMobModel,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    n_epochs: int = 100,
    learning_rate: float = 1e-3,
    patience: int = 10,
    teacher_forcing_ratio: float = 0.5
):
    """
    Treina o modelo HuMob com early stopping e teacher forcing.
    
    teacher_forcing_ratio: probabilidade de usar ground truth vs predições durante treino
                          É como decidir quando "dar a resposta" vs deixar o modelo "chutar"
    """
    
    # Setup do treinamento
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    criterion = HuMobLoss(model.destination_head.centers)
    
    # Métricas para acompanhar
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    
    print(f"Treinando modelo por {n_epochs} épocas...")
    print(f"Device: {device}")
    print(f"Parâmetros treináveis: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    
    for epoch in range(n_epochs):
        # === TREINAMENTO ===
        model.train()
        train_loss_epoch = 0
        train_coord_loss_epoch = 0
        
        train_pbar = tqdm(train_loader, desc=f'Época {epoch+1}/{n_epochs} [Treino]')
        
        for batch_idx, batch in enumerate(train_pbar):
            uid, d, t, city, poi, coords_seq, target_coords = [b.to(device) for b in batch]
            
            optimizer.zero_grad()
            
            # Decide se usa teacher forcing (durante treino sequencial)
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_ratio
            
            if target_coords.size(1) == 1:  # Single step prediction
                pred_coords = model.forward_single_step(uid, d, t, city, poi, coords_seq)
                target = target_coords.squeeze(1)  # Remove dimensão do step
            else:  # Multi-step prediction
                pred_coords = model.rollout_predictions(
                    uid, d, t, city, poi, coords_seq,
                    n_steps=target_coords.size(1),
                    use_predictions=not use_teacher_forcing
                )
                target = target_coords
            
            # Calcula loss
            loss, coord_loss, cluster_loss = criterion(pred_coords, target)
            
            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            
            # Accumula métricas
            train_loss_epoch += loss.item()
            train_coord_loss_epoch += coord_loss.item()
            
            # Atualiza progress bar
            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Coord': f'{coord_loss.item():.4f}',
                'LR': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })
        
        # === VALIDAÇÃO ===
        model.eval()
        val_loss_epoch = 0
        val_coord_loss_epoch = 0
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Época {epoch+1}/{n_epochs} [Val]')
            
            for batch in val_pbar:
                uid, d, t, city, poi, coords_seq, target_coords = [b.to(device) for b in batch]
                
                if target_coords.size(1) == 1:
                    pred_coords = model.forward_single_step(uid, d, t, city, poi, coords_seq)
                    target = target_coords.squeeze(1)
                else:
                    pred_coords = model.rollout_predictions(
                        uid, d, t, city, poi, coords_seq,
                        n_steps=target_coords.size(1),
                        use_predictions=True  # Sempre usa predições na validação
                    )
                    target = target_coords
                
                loss, coord_loss, _ = criterion(pred_coords, target)
                
                val_loss_epoch += loss.item()
                val_coord_loss_epoch += coord_loss.item()
                
                val_pbar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
        
        # Médias da época
        avg_train_loss = train_loss_epoch / len(train_loader)
        avg_val_loss = val_loss_epoch / len(val_loader)
        avg_train_coord = train_coord_loss_epoch / len(train_loader)
        avg_val_coord = val_coord_loss_epoch / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Salva melhor modelo
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': avg_val_loss,
                'train_loss': avg_train_loss
            }, 'best_humob_model.pth')
        else:
            patience_counter += 1
        
        # Print estatísticas da época
        print(f'\nÉpoca {epoch+1}:')
        print(f'  Train Loss: {avg_train_loss:.4f} (Coord: {avg_train_coord:.4f})')
        print(f'  Val Loss:   {avg_val_loss:.4f} (Coord: {avg_val_coord:.4f})')
        print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}')
        print(f'  Fusion weights: w_r={model.weighted_fusion.w_r.item():.3f}, w_e={model.weighted_fusion.w_e.item():.3f}')
        
        if patience_counter >= patience:
            print(f'\nEarly stopping! Sem melhoria por {patience} épocas.')
            break
    
    # Plot das curvas de treinamento
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Curvas de Treinamento')
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot([model.weighted_fusion.w_r.item()], [model.weighted_fusion.w_e.item()], 'ro', markersize=10)
    plt.xlabel('w_r (peso estático)')
    plt.ylabel('w_e (peso dinâmico)')
    plt.title('Pesos Finais da Fusão')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return train_losses, val_losses


# Função para usar no seu notebook
def setup_and_train():
    """
    Função principal que você pode chamar no seu notebook.
    Substitui a parte que está faltando.
    """
    # 1. Calcula centros dos clusters (seu código já faz isso)
    cluster_centers = compute_hdbscan_centers(
        parquet_file,
        city_letter="D",
        day_threshold=60,
        chunk_size=50_000,
        min_cluster_size=200
    ).to(device)
    
    # 2. Cria dataset corrigido
    train_ds = ParquetCityDDataset(
        parquet_file, 
        city_list=["D"], 
        chunk_size=2500,
        sequence_length=24,  # 24 slots de história
        prediction_steps=1   # Prediz 1 passo à frente
    )
    train_loader = DataLoader(train_ds, batch_size=32, num_workers=2, shuffle=False)
    
    # 3. Cria modelo completo
    model = HuMobModel(
        n_users=n_users_D,
        n_days=75,
        n_slots=48,
        n_cities=4,
        cluster_centers=cluster_centers,
        emb_dim=10,
        poi_in_dim=85,
        poi_out_dim=10,
        lstm_hidden=10,
        fusion_dim=20
    ).to(device)
    
    # 4. Treina (você pode dividir em train/val se quiser)
    train_losses, val_losses = train_humob_model(
        model=model,
        train_loader=train_loader,
        val_loader=train_loader,  # Por enquanto usa mesmo dataset
        device=device,
        n_epochs=50,
        learning_rate=1e-3,
        patience=10
    )
    
    return model, train_losses, val_losses